## HW 1 ##

In [0]:
import numpy as np
from scipy import linalg
import random

### Problem 2 ###

**Some definitions**

Let 

$$M\in\mathbb{R}^{n\times n}$$ be an arbitrary matrix.  

Let $$p(x)=a_0 + a_1 x + a_2 x^2 + \ldots + a_n x^n \in\mathbb{R}[x]$$ be an arbitrary polynomial of less or equal to $n$.

The above polynomial can be used to define a matrix function that takes matrices as input and outputs matrices as follows: 

$$p(M) = a_0 I + a_1 M + \ldots + a_n M^n,$$ 

that is, each monomial $x^k$ is substituted by the corresponding matrix power $M^k$.

We say that a polynomial $p(x)$ annihilates a matrix $M\in\mathbb{R}^{n\times n}$ iff $p(M)=\boldsymbol{0}$, where $\boldsymbol{0}$ is the zero matrix.

**Task**

The task is to write a function ```annihilate_poly``` that takes as input an arbitrary square numpy array $M$ and outputs a vector whose cofficients are the coefficients of a (non-trivial) polynomial that annihilates $M$.  One-trivial means that its is not the zero polynomial which maps every matrix to the zero matrix.

**Hint**

You can reduce the problem to finding a linear dependance relationship between the $n+1$ vectors 

$$\mathrm{vec}(I), \mathrm{vec}(M), \mathrm{vec}(M^2),\ldots,\mathrm{vec}(M^n)\in\mathrm{R}^{n^2}.$$



The operation $\mathrm{vec}$ turns a square matrix $M\in\mathbb{R}^{n\times n}$ into a vector $v\in\mathbb{R}^{n^2}$ by first listing the entries of the first row, then those of the second row etc.

Update: 

To solve this problem, you have to compute the null space of the matrix $A\in \mathbb{R}^{n^2\times (n+1)}$ whose columns are the vectors $\mathrm{vec}(M^k)$ for $k\in\{0,\ldots,n\}$.


(This is not needed: 

If you don't remeber how to compute the find a linear dependance relationship, check out this stackoverflow post: https://math.stackexchange.com/questions/2198960/finding-linear-dependence-relation

You can use https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html to solve the resulting matrix equation.)

In [0]:
def annihilate_poly(M):
  '''
  Return a basis for all the vectors whose coefficients form a non-trivial polynomial that annihilate the matrix M
  '''

  M = np.matrix(M)

  # Check that M is a square matrix
  if M.shape[0] != M.shape[1]:
    raise ValueError('Not a square matrix')

  n = M.shape[0]

  # Build matrix A, whose columns are the vectors vec(M^k) for 0 <= k <= n
  A = np.empty((n**2, n+1), dtype=int)
  for i in range(n+1):
    col = (M**i).reshape((n**2,))
    A[:, i] = col

  # This will return a basis for the null space of A
  # From this, we could constuct an infinite number of annihilating polynomials
  ps = linalg.null_space(A)

  # However, we only need a single annihilating polynomial
  # Since any of the vectors from this basis will form annihilating polynomials, I can simply return any of the vectors 
  
  # Randomly choose a vector from the basis and return it
  rand = random.randint(0, ps.shape[1]-1)
  return ps[:, rand]

Below I will demonstrate that my function can successfully find annihilating polynomials

In [5]:
M = np.matrix([[1,0], [0,1]])
print('M:')
print(M)
print()

p = annihilate_poly(M)
print('Annihilating polynomial coefficients:')
print(p)
print()

M:
[[1 0]
 [0 1]]

Annihilating polynomial coefficients:
[ 0.         -0.70710678  0.70710678]



To prove my function is outputting annihilating polynomials, I create a function which plugs in matrix M into the polynomial and checks whether the result is equal to the zero matrix.

In [0]:
def check_annihilates(M, p):
  '''
  Checks whether a polynomial p annihilates the matrix M
  '''

  result = 0

  # Creating a string to represent the polynomial
  polynomial = 'p(x) = '

  # Plug matrix M into p
  for i, coef in enumerate(p):
    result += coef * (M**i)

    # Forming the polynomial string 
    symbol = ' + ' if coef >= 0 else ' - '
    str_coef = str(coef).strip('-')
    polynomial += '{}{}x^{}'.format(symbol, str_coef, i) if i != 0 else '{}x^{}'.format(coef, i)
  
  # Check whether the result is equal to the zero matrix
  if np.array_equal(np.round(result, 5), np.zeros_like(result)):
    print('Success! {} is an annihilating polynomial \n'.format(polynomial))
  else:
    print('Failure! {} is not an annihilating polynomial \n'.format(polynomial))

In [7]:
check_annihilates(M, p)

Success! p(x) = 0.0x^0 - 0.7071067811865475x^1 + 0.7071067811865476x^2 is an annihilating polynomial 



To ensure my function works properly, I will now generate a random matrix and use my function to generate an annihilating polynomial, then prove it is correct.

In [8]:
# Generate a random square matrix, anywhere from shape (1,1) to (7,7)
n = random.randint(1,7)
M = np.matrix([np.random.randint(10, size = n) for _ in range(n)])
print('M:')
print(M)
print()

# Find annihilating polynomial
p = annihilate_poly(M)
print('Annihilating polynomial coefficients:')
print(p)
print()

check_annihilates(M, p)

M:
[[2 9 9 3 1]
 [4 7 0 2 7]
 [0 7 9 6 7]
 [3 4 2 9 1]
 [3 9 6 8 3]]

Annihilating polynomial coefficients:
[-9.30624693e-01  3.64086909e-01  2.08712241e-03 -3.64086909e-02
  6.95707470e-03 -2.31902490e-04]

Success! p(x) = -0.9306246927939937x^0 + 0.36408690945788297x^1 + 0.0020871224109648263x^2 - 0.036408690945250656x^3 + 0.006957074702916784x^4 - 0.00023190249009724662x^5 is an annihilating polynomial 



**Task**

Write a function ```annihilate_min_deg_poly``` that computes a non-trivial polynomial that annihilates a given square matrix and has the smallest possible degree.  Recall that a polynomial $p(x)$ has degree $d$ if the coefficient $a_{d+1}=\ldots=a_n=0$.

Recall that my function ```annihilate_poly``` creates a basis for all the vectors whose coefficients form a polynomial that annihilates the matrix M, and then randomly returns one of the vectors from this basis. 

To find the annihilating polynomial with the smallest degree, rather than choosing a vector directly from the basis, I simply need to leverage this basis to generate the smallest possible degree polynomial. This will require a slight change to my original function.

In [0]:
def annihilate_min_deg_poly(M):
  M = np.matrix(M)

  # Check that M is a square matrix
  if M.shape[0] != M.shape[1]:
    raise ValueError('Not a square matrix')

  n = M.shape[0]

  # Build matrix A, whose columns are the vectors vec(M^k) for 0 <= k <= n
  A = np.empty((n**2, n+1), dtype=int)
  for i in range(n+1):
    col = (M**i).reshape((n**2,))
    A[:, i] = col

  # This will return a basis for the null space of A
  ps = linalg.null_space(A)

  # Construct the smallest possible degree annihilating polynomial from the basis
  lowest_degree_p = ps[:, 0]

  return lowest_degree_p

Testing it out...

In [10]:
n = random.randint(1,6)
M = np.matrix([np.random.randint(10, size = n) for _ in range(n)])
M = np.matrix([[1,0], [0,1]])
M = np.matrix([[1,2,1,0], [-2,1,0,1], [0,0,1,2], [0,0,-2,1]])
print('M:')
print(M)
print()
print(np.poly(M))
lowest_degree_p = annihilate_min_deg_poly(M)
print('Annihilating polynomial coefficients:')
print(lowest_degree_p)
print()

check_annihilates(M, lowest_degree_p)

M:
[[ 1  2  1  0]
 [-2  1  0  1]
 [ 0  0  1  2]
 [ 0  0 -2  1]]

[  1.  -4.  14. -20.  25.]
Annihilating polynomial coefficients:
[-0.71052553  0.56842042 -0.3978943   0.11368408 -0.02842102]

Success! p(x) = -0.7105255285040057x^0 + 0.5684204228032044x^1 - 0.39789429596224307x^2 + 0.11368408456064083x^3 - 0.028421021140160208x^4 is an annihilating polynomial 

